In [1]:
# rmsoldie genetic algorithms
import sys
import matplotlib.pyplot as plt
import moviepy.editor as mvp
import numpy as np
import os
import random
import time
import tqdm
from matplotlib.colors import to_rgb
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
from PIL import Image, ImageDraw, ImageTk
from enum import Enum, EnumMeta, auto
from IPython.display import display, Video, HTML, clear_output

In [3]:
def clamp(lower, higher, val):
    if val < lower:
        val = lower
    elif val > higher:
        val = higher
    return val

In [ ]:
# class Output(Enum):
#     pass
# class AllOutputs(Output):
#     EMPTY = 0
#     ALL_OUTPUTS = 1
# class NoOutput(Output):
#    UP_INPUT = 10
#    RIGHT_INPUT = 11
#    DOWN_INPUT = 12
#    LEFT_INPUT = 13
# class SideOutputs(Output):
#     UP_INPUT = 20
#     RIGHT_INPUT = 21
#     DOWN_INPUT = 22
#     LEFT_INPUT = 23
# class ThreeOutputs(Output):
#     UP_INPUT = 30
#     RIGHT_INPUT = 31
#     DOWN_INPUT = 32
#     LEFT_INPUT = 33
    
class Vesicle():
    def __init__(self, grid):
        # dir: (wavebool, gapsize)
        directions = ["N", "E", "S", "W"]
        self.empty_table = [(direct, False, 0) for direct in directions]
        self.input_table = self.empty_table.copy()
        self.output_table = self.empty_table.copy()
        self.state_val = 0
        self.grid = grid
    
    def update(self):
        # get new inputs
        # calc state
        pass
        
    def calc_state(self):
        global gap_threshold
        inp = self.input_table
        stimed_directs = []
        
        for (direct, wavebool, gapsize) in inp:
            if wavebool == True and gapsize > gap_threshold:
                stimed_directs.append(direct)
                
        num_stimed_directs = len(stimed_directs)
        if num_stimed_directs==0:
            self.output_table.
            
        elif num_stimed_directs==1:
            # 
            
        elif num_stimed_directs==2:
            
        elif num_stimed_directs==3:
            # whichever is not in the list gets outputted
            
        elif num_stimed_directs==4:
            # no output
            
                
                
                
                
                
                if direct=="N":
                    pass
                elif direct=="E":
                    pass
                elif direct=="S":
                    pass
                else direct=="W":
                    pass


In [8]:
def build_lookup_table():
    table=dict()
    temp = []

    temp.append(())
    temp.append(("N"))
    temp.append(("E"))
    temp.append(("S"))
    temp.append(("W"))
    temp.append(("N", "E"))
    temp.append(("N", "S"))
    temp.append(("N", "W"))
    temp.append(("E", "S"))
    temp.append(("E", "W"))
    temp.append(("W", "S"))
    temp.append(("N", "E", "S"))
    temp.append(("N", "E", "W"))
    temp.append(("N", "S", "W"))
    temp.append(("E", "S", "W"))
    temp.append(("N", "E", "S", "W"))
    
    for i in range(len(temp)):
        elem = temp[i]
        table[elem] = i
    
    return table

table = build_lookup_table()
table

{(): 0,
 'N': 1,
 'E': 2,
 'S': 3,
 'W': 4,
 ('N', 'E'): 5,
 ('N', 'S'): 6,
 ('N', 'W'): 7,
 ('E', 'S'): 8,
 ('E', 'W'): 9,
 ('W', 'S'): 10,
 ('N', 'E', 'S'): 11,
 ('N', 'E', 'W'): 12,
 ('N', 'S', 'W'): 13,
 ('E', 'S', 'W'): 14,
 ('N', 'E', 'S', 'W'): 15}

In [ ]:
def get_opposite_dir(dir):
    opp_dir = (dir+2)%4
    return opp_dir

def get_neigh_input(dir, cell_tuple: tuple[Output, list]):
    dir = get_opposite_dir(dir)
    _, out_arr = cell_tuple
    if dir in out_arr:
        return True
    else:
        return False
    
def calc_new_values(cell_tuple: tuple[Output, list]):
    # get all 4 neigh values
    enum, out_arr = cell_tuple
    if enum in AllOutputs._member_names_:
        # handle empty cell
        # handle all outputs
        pass
    
    input_dir = enum.value%10
    # ooh how to handle this...
    pass

def update_vesicles(state_arr):
    num_rows, num_columns = np.shape(state_arr)
    new_array = []
    for x in range(num_rows):
        for y in range(num_columns):
            # decide new value
                   
            
            new_array[x, y]
    return new_array

def get_vals_to_print(state_arr: list):
    num_rows, num_columns = np.shape(state_arr)
    out = ""
    for x in range(num_rows):
        for y in range(num_columns):
            out+=" "+state_arr[x, y]
        out+="\n"
    return out

In [ ]:
h_in_vesicles = 2
w_in_vesicles = 3
frames = 5
gap_threshold = 0

stim_row, stim_column = 1, 1

In [ ]:
# ------------ main ------------ #
# state_arr = np.full((h_in_vesicles, w_in_vesicles), (AllOutputs.EMPTY, []))
state_arr = np.full((h_in_vesicles, w_in_vesicles), None)
state_arr[stim_row, stim_column] = (AllOutputs.ALL_OUTPUTS, [0, 1, 2, 3])

print("shape of image array:", state_arr.shape)

for frame in range(frames):
    state_arr = update_vesicles(state_arr)
    clear_output(wait=True)
    display(get_vals_to_print(state_arr))
    time.sleep(1)

shape of image array: (3, 3)


TypeError: list indices must be integers or slices, not tuple